## Markowitz Model for Portfolio Optimization

The objective of this project is to formulate and solve the Markowitz model for portfolio optimization using Python. The Markowitz model aims to optimize the allocation of capital among a set of assets to minimize the overall risk (variance) of the portfolio for a given level of expected return. This is achieved by solving a quadratic optimization problem. In this project we'll be using the qpsolvers package to solve this optimization problem and plot the efficient frontier, which shows the trade-off between risk and return for different levels of expected returns.

### Model Description

We consider a scenario where we have some capital to invest across $n$ different assets. The goal is to determine the optimal allocation of capital to minimize the risk while achieving a specified return. 

Here is a brief description of the model:

- **Asset Allocation**: Let $x_j$ represent the fraction of total capital invested in asset $j$ for $j = 1, \ldots, n$.
- **Returns**: The annual (or daily) return of each asset is modeled as a random variable $\xi_j$.
- **Expected Returns**: The expected return of each asset is denoted by $\mu_j = E[\xi_j]$.
- **Covariance Matrix**: The covariance between returns of assets $i$ and $j$ is represented by $C_{ij} = E[(\xi_i - \mu_i)(\xi_j - \mu_j)]$.

Given these, the annual return of the portfolio is defined as the weighted sum of the expected returns: $r = \mu^T x$ where $\mu = (\mu_1, \ldots, \mu_n)$ is the vector of expected returns and $x = (x_1, \ldots, x_n)^T$ is the vector of capital allocations.


### Optimization Problem

The objective is to minimize the variance of the portfolio, which can be expressed as:

Minimize: 
* $\quad x^T C x$

Subject to: 
* $\mu^T x = r$
* $e^T x = 1, \quad \text{where } e = (1, \ldots, 1)^T$
* $x \geq 0$

### Requirements

In [1]:
import numpy as np
import random
from qpsolvers import solve_qp
import matplotlib.pyplot as plt

### Setup

For the setup, we will first input the last 2 digits of my registration number into `dig1` and `dig2` and use them to calculate `dummyrepetitions` which is a value used to initialize the random number generator state, ensuring that the random numbers generated later are reproducible. We'll set `n` equal to 10 as that is the number of different assets considered in the optimization problem. Then we can loop through the `dummyrepetitions` which in this case is 19 and generate a number either 0 or 1 in each iteration. This step initializes the random number generator state to ensure that the subsequent random number generations are influenced by these initializations, contributing to the randomness in the generated data.


In [2]:
# registation number
dig1 = 1
dig2 = 9
dummyrepetitions = 10 * dig1 + dig2

# parameters
n = 10

In [3]:
# generate data
for _ in range(dummyrepetitions):
    dummy = random.uniform(0, 1)

For the correlation matrix, we'll initialize an `n` by `n` matrix (10 x 10 in this case) filled with zeroes and then use the correlation formula to fill the matrix where the correlation between assets decreases with the distances between their indices.

In [4]:
# create correlation matrix
Corr = np.array([[0] * n for _ in range(n)], dtype=float)
print(Corr)

for i in range(n):
    for j in range(n):
        Corr[i][j] = (-1) ** abs(i - j) / (abs(i - j) + 1)
print(Corr)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 1.         -0.5         0.33333333 -0.25        0.2        -0.16666667
   0.14285714 -0.125       0.11111111 -0.1       ]
 [-0.5         1.         -0.5         0.33333333 -0.25        0.2
  -0.16666667  0.14285714 -0.125       0.11111111]
 [ 0.33333333 -0.5         1.         -0.5         0.33333333 -0.25
   0.2        -0.16666667  0.14285714 -0.125     ]
 [-0.25        0.33333333 -0.5         1.         -0.5         0.33333333
  -0.25        0.2        -0.16666667  0.14285714]
 [ 0.2        -0.25        0.33333333 -0.5         1.         -0.5
   0.33333333 -0.25        0.2        -0.16666667]
 [-0.16666667  0.2        -0.25        0.33333333 -0.5         1

We will initialize `ssigma` (standard deviations) and `mmu` (expected returns) as column vectors of zeroes with `n` rows. The initial values for `ssigma` and `mmu` will be 2 and 3 respectively. To populate the two vectors we'll use a loop such that each subsequent element of `ssigma` is the previous element plus a random number between 0-2 and each subsequent element of `mmu` is the previous element plus 1.

These vectors represent the standard deviations and expected returns of the 10 assets.

In [5]:
# initialize std dev and ER
ssigma = np.array([[0] * 1 for _ in range(n)], dtype = float)
mmu = np.array([[0] * 1 for _ in range(n)], dtype = float)

ssigma[0] = 2
mmu[0] = 3

print(ssigma)
print(mmu)

for i in range(n - 1):
    ssigma[i + 1] = ssigma[i] + 2 * random.uniform(0, 1)
    mmu[i + 1] = mmu[i] + 1

print(ssigma)
print(mmu)

[[2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[3.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[ 2.        ]
 [ 3.93477192]
 [ 4.39186639]
 [ 5.54252813]
 [ 5.95076271]
 [ 6.38398646]
 [ 7.22004364]
 [ 8.32299455]
 [ 9.46877824]
 [11.03470362]]
[[ 3.]
 [ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]
 [11.]
 [12.]]


For the covariance matrix we'll do the same as before and initialize a 10x10 matrix filled with zeroes. Then we can fill the diagonal values with the values from the `ssigma` vector we created previously. To calculate the preliminary covariance matrix we'll use matrix multiplication for `ddiag`, `Corr`, and `ddiag` again. This step effectively scales the correlation matrix by the standard deviations. To ensure symmetry we can take the average of C2 and its transpose. It also helps to guarantee that the resulting matrix is positive semi-definite.

In [6]:
# create covariance matrix
ddiag = np.array([[0] * n for _ in range(n)], dtype = float)
np.fill_diagonal(ddiag, ssigma)
print(ddiag)

C2 = np.matmul(np.matmul(ddiag,Corr), ddiag)
print(C2)

C = 0.5 * (C2 + C2.T)
print(C)

[[ 2.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          3.93477192  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          4.39186639  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          5.54252813  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          5.95076271  0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          6.38398646
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   7.22004364  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          8.32299455  0.          0.        ]
 [ 0.          0.          0.          0.          0.   

The last part of the setup is to convert the `mmu` vector from a *(n , 1)* shape to a 1D array of shape *(n, )*.

In [7]:
# ER vector
print(mmu)
mu = mmu.flatten()
print(mu)

[[ 3.]
 [ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]
 [11.]
 [12.]]
[ 3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


### Task 1

For Task 1, the objective is to solve the Markowitz optimization problem for a range of target return values given. The goal is to find the portofolio weights that minimize the variance for each given level of expected return.

In [8]:
# define r values
r_values = np.arange(2.00, 9.25, 0.25)
print(r_values)
print(r_values.size)

[2.   2.25 2.5  2.75 3.   3.25 3.5  3.75 4.   4.25 4.5  4.75 5.   5.25
 5.5  5.75 6.   6.25 6.5  6.75 7.   7.25 7.5  7.75 8.   8.25 8.5  8.75
 9.  ]
29


In [9]:
def t1(r_values):
    sigmas = []
    mus = []

    for r in r_values:
        # quadprog setup
        P = C
        q = np.zeros(n)
        G = -np.eye(n)
        h = np.zeros(n)
        A = np.vstack([mu, np.ones(n)])
        b = np.array([r, 1])

        # debug information
        print(f"Solving for r = {r}")
        print("P:", P)
        print("q:", q)
        print("G:", G)
        print("h:", h)
        print("A:", A)
        print("b:", b)

        # solve qp problem
        x = solve_qp(P, q, G, h, A, b, solver='quadprog')
        if x is None:
            print(f"No solution found for r = {r}")
            continue

        sigma = np.sqrt(np.dot(x.T, np.dot(C, x)))
        mu_val = np.dot(mu.T, x)

        sigmas.append(sigma)
        mus.append(mu_val)

    return sigmas, mus

# solve
sigmas, mus = t1(r_values)

Solving for r = 2.0
P: [[  4.          -3.93477192   2.92791093  -2.77126407   2.38030508
   -2.12799549   2.06286961  -2.08074864   2.10417294  -2.20694072]
 [ -3.93477192  15.48243006  -8.64049627   7.26952802  -5.85372351
    5.02390613  -4.73487083   4.67844075  -4.65718534   4.824338  ]
 [  2.92791093  -8.64049627  19.28849039 -12.17102151   8.71165158
   -7.00940389   6.3418934   -6.09224667   5.94080127  -6.057868  ]
 [ -2.77126407   7.26952802 -12.17102151  30.71961812 -16.49113487
   11.79447485 -10.00432375   9.2260863   -8.7468283    8.73716504]
 [  2.38030508  -5.85372351   8.71165158 -16.49113487  35.41157685
  -18.99479428  14.32158882 -12.38204141  11.2692905  -10.94415048]
 [ -2.12799549   5.02390613  -7.00940389  11.79447485 -18.99479428
   40.7552831  -23.0463304   17.71129484 -15.11213802  14.0890797 ]
 [  2.06286961  -4.73487083   6.3418934  -10.00432375  14.32158882
  -23.0463304   52.12903011 -30.04619193  22.78833069 -19.91776042]
 [ -2.08074864   4.67844075  -6.

In [ ]:
# plot
plt.figure(figsize=(10, 6))
plt.plot(sigmas, mus, marker='o')
plt.xlabel('σ (Risk)')
plt.ylabel('μ (Return)')
plt.title('Efficient Frontier')
plt.grid(True)
plt.show()

In [ ]:
def t2(r_values):
    sigmas = []
    mus = []

    for r in r_values:
        # Quadratic programming setup
        P = C
        q = np.zeros(n)
        G = -np.eye(n)
        h = np.zeros(n)
        A = np.vstack([mu, np.ones(n)])
        b = np.array([r, 1])

        # Modify the equality constraint to an inequality constraint
        G = np.vstack([G, np.ones(n)])
        h = np.hstack([h, [1]])

        # Solve QP problem
        x = solve_qp(P, q, G, h, A[:-1], b[:-1], solver='quadprog')
        if x is None:
            print(f"No solution found for r = {r}")
            continue

        sigma = np.sqrt(np.dot(x.T, np.dot(C, x)))
        mu_val = np.dot(mu.T, x)

        sigmas.append(sigma)
        mus.append(mu_val)

    return sigmas, mus

# Solve task 2
sigmas_2, mus_2 = t2(r_values)

# Plot results for Task 2
plt.figure(figsize=(10, 6))
plt.plot(sigmas_2, mus_2, marker='o')
plt.xlabel('σ (Risk)')
plt.ylabel('μ (Return)')
plt.title('Efficient Frontier (Task 2)')
plt.grid(True)
plt.show()

In [ ]:
def t3(r_values):
    sigmas = []
    mus = []

    for r in r_values:
        # Quadratic programming setup
        P = C
        q = np.zeros(n)
        G = -np.eye(n)
        h = np.zeros(n)
        A = np.vstack([mu, np.ones(n)])
        b = np.array([r, 1])

        # Modify the equality constraint to an inequality constraint
        G = np.vstack([G, -mu])
        h = np.hstack([h, -r])

        # Solve QP problem
        x = solve_qp(P, q, G, h, A[-1:], b[-1:], solver='quadprog')
        if x is None:
            print(f"No solution found for r = {r}")
            continue

        sigma = np.sqrt(np.dot(x.T, np.dot(C, x)))
        mu_val = np.dot(mu.T, x)

        sigmas.append(sigma)
        mus.append(mu_val)

    return sigmas, mus

# Solve task 3
sigmas_3, mus_3 = t3(r_values)

# Plot results for Task 3
plt.figure(figsize=(10, 6))
plt.plot(sigmas_3, mus_3, marker='o')
plt.xlabel('σ (Risk)')
plt.ylabel('μ (Return)')
plt.title('Efficient Frontier (Task 3)')
plt.grid(True)
plt.show()

In [ ]:
def t4(r_values):
    sigmas = []
    mus = []

    for r in r_values:
        # Quadratic programming setup
        P = C
        q = np.zeros(n)
        A = np.vstack([mu, np.ones(n)])
        b = np.array([r, 1])

        # Remove the non-negativity constraint
        G = None
        h = None

        # Solve QP problem
        x = solve_qp(P, q, G, h, A, b, solver='quadprog')
        if x is None:
            print(f"No solution found for r = {r}")
            continue

        sigma = np.sqrt(np.dot(x.T, np.dot(C, x)))
        mu_val = np.dot(mu.T, x)

        sigmas.append(sigma)
        mus.append(mu_val)

    return sigmas, mus

# Solve task 4
sigmas_4, mus_4 = t4(r_values)

# Plot results for Task 4
plt.figure(figsize=(10, 6))
plt.plot(sigmas_4, mus_4, marker='o')
plt.xlabel('σ (Risk)')
plt.ylabel('μ (Return)')
plt.title('Efficient Frontier (Task 4)')
plt.grid(True)
plt.show()